## Watch the trained agent 

In [1]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="Tennis.app") # Works on a mac

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# size of each action
# actions between -1 and 1
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)
# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [2]:
from buffer import ReplayBuffer
from maddpg import MADDPG
import torch
import numpy as np
from tensorboardX import SummaryWriter
import os
from utilities import transpose_list, transpose_to_tensor
from collections import deque
from itertools import count

maddpg = MADDPG(seed=100)

state_dict = torch.load("episode-2500.pt", map_location='cpu')
for i in range(2):
    maddpg.maddpg_agent[i].actor.load_state_dict(state_dict[i]["actor_params"])
    maddpg.maddpg_agent[i].actor_optimizer.load_state_dict(state_dict[i]["actor_optim_params"])
    maddpg.maddpg_agent[i].critic.load_state_dict(state_dict[i]["critic_params"])    
    maddpg.maddpg_agent[i].critic_optimizer.load_state_dict(state_dict[i]["critic_optim_params"]) 
    
state = env.reset()
env_info = env.reset(train_mode=False)[brain_name]  # reset the environment
state = env_info.vector_observations # get the current state (for each agent)
for t in count(): 
    actions = maddpg.act(torch.tensor(state,dtype=torch.float))
    action = torch.stack(actions).detach().numpy()
    env_info = env.step(action)[brain_name]
    next_states = env_info.vector_observations
    state = next_states

KeyboardInterrupt: 